<a href="https://colab.research.google.com/github/mabataki2/AI-Class/blob/main/Week10/Simple_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler # 스케일러 임포트
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau
# --- 1. 561-Feature 데이터 로드 함수 (스케일링 및 3D 변환 포함) ---
def load_feature_dataset(prefix=''):
    """
    UCI HAR Dataset의 X_train/X_test.txt (561개 피처) 파일을 로드합니다.
    - StandardScaler를 적용합니다.
    - CNN 입력을 위해 (N, 561, 1) 형태로 변환합니다.
    """

    # 561 피처 데이터 로드
    # prefix에 '/content/drive/MyDrive/UCI HAR Dataset/'와 같은 기본 경로가 포함됩니다.
    trainX = pd.read_csv(prefix + 'train/X_train.txt', header=None, delim_whitespace=True).values
    trainy = pd.read_csv(prefix + 'train/y_train.txt', header=None, delim_whitespace=True).values

    testX = pd.read_csv(prefix + 'test/X_test.txt', header=None, delim_whitespace=True).values
    testy = pd.read_csv(prefix + 'test/y_test.txt', header=None, delim_whitespace=True).values

    # 스케일링
    scaler = StandardScaler()
    trainX = scaler.fit_transform(trainX)
    testX = scaler.transform(testX)

    # CNN 사용을 위한 3D 변환 (N, 561) -> (N, 561, 1)
    trainX = np.expand_dims(trainX, axis=-1)
    testX = np.expand_dims(testX, axis=-1)

    # Y 레이블 처리 (1~6 -> 0~5) 및 원-핫 인코딩
    trainy = trainy - 1
    testy = testy - 1
    trainy_one_hot = to_categorical(trainy)
    testy_one_hot = to_categorical(testy)

    print(f"로드 및 변환된 561-Feature 데이터 Shape:")
    print(f"Train X: {trainX.shape}, Train Y (One-Hot): {trainy_one_hot.shape}")
    print(f"Test X: {testX.shape}, Test Y (One-Hot): {testy_one_hot.shape}")

    return trainX, trainy_one_hot, testX, testy_one_hot

In [5]:
# -------------------------------------------------------------

# --- 2. 데이터 로드 및 분할 ---
prefix_path = '/content/drive/MyDrive/UCI HAR Dataset/'

trainX_feat, trainy_one_hot_feat, testX_feat, testy_one_hot_feat = load_feature_dataset(prefix=prefix_path)

# 훈련/검증 데이터 분할
X_train_f, X_val_f, y_train_f, y_val_f = train_test_split(
    trainX_feat, trainy_one_hot_feat, test_size=0.2, random_state=100
)

# 모델 입력 차원 정의
n_timesteps_f, n_features_f = X_train_f.shape[1], X_train_f.shape[2] # 561, 1
n_outputs_f = y_train_f.shape[1] # 6

print(f"\n모델 입력 정의: Timesteps={n_timesteps_f}, Features={n_features_f}, Outputs={n_outputs_f}")

/tmp/ipython-input-4202150923.py:19: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  trainX = pd.read_csv(prefix + 'train/X_train.txt', header=None, delim_whitespace=True).values
/tmp/ipython-input-4202150923.py:20: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  trainy = pd.read_csv(prefix + 'train/y_train.txt', header=None, delim_whitespace=True).values
/tmp/ipython-input-4202150923.py:22: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  testX = pd.read_csv(prefix + 'test/X_test.txt', header=None, delim_whitespace=True).values


로드 및 변환된 561-Feature 데이터 Shape:
Train X: (7352, 561, 1), Train Y (One-Hot): (7352, 6)
Test X: (2947, 561, 1), Test Y (One-Hot): (2947, 6)

모델 입력 정의: Timesteps=561, Features=1, Outputs=6


/tmp/ipython-input-4202150923.py:23: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  testy = pd.read_csv(prefix + 'test/y_test.txt', header=None, delim_whitespace=True).values


In [6]:
# --- 3. 561-Feature용 CNN 모델 정의 ---
def create_cnn_on_features(timesteps, features, outputs):
    model = Sequential()

    # Layer 1
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(timesteps, features)))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))

    # Layer 2
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))

    # Layer 3
    model.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))

    model.add(Flatten())

    # Dense Layers
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    model.add(Dense(outputs, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# 모델 생성
feature_cnn_model = create_cnn_on_features(n_timesteps_f, n_features_f, n_outputs_f)
feature_cnn_model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 559, 64)        │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 559, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 279, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 277, 128)       │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 277, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 138, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 136, 256)       │        98,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 136, 256)       │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 68, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 17408)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     4,456,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │         1,542 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,584,582 (17.49 MB)

 Trainable params: 4,583,174 (17.48 MB)

 Non-trainable params: 1,408 (5.50 KB)

In [7]:
# --- 4. 모델 훈련 ---
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001, verbose=1)

print("\n--- 모델 훈련 시작 (561-Feature CNN) ---")
history_feat_cnn = feature_cnn_model.fit(
    X_train_f, y_train_f,
    epochs=30, # 충분한 학습을 위해 에포크 증가
    batch_size=64,
    validation_data=(X_val_f, y_val_f),
    verbose=1,
    callbacks=[reduce_lr]
)



--- 모델 훈련 시작 (561-Feature CNN) ---
Epoch 1/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 15s 68ms/step - accuracy: 0.7832 - loss: 0.7073 - val_accuracy: 0.1652 - val_loss: 6.6360 - learning_rate: 0.0010
Epoch 2/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9723 - loss: 0.0848 - val_accuracy: 0.1652 - val_loss: 5.6985 - learning_rate: 0.0010
Epoch 3/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9848 - loss: 0.0505 - val_accuracy: 0.2012 - val_loss: 2.0907 - learning_rate: 0.0010
Epoch 4/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9788 - loss: 0.0633 - val_accuracy: 0.8797 - val_loss: 0.3273 - learning_rate: 0.0010
Epoch 5/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9897 - loss: 0.0295 - val_accuracy: 0.9742 - val_loss: 0.0798 - learning_rate: 0.0010
Epoch 6/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9970 - loss: 0.0130 - val_accuracy: 0.9762 - val_loss: 0.0654 - learning_rate: 0.0010
Epoch 7/30
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accu

In [8]:
# --- 5. 모델 평가 ---
loss_f, accuracy_f = feature_cnn_model.evaluate(testX_feat, testy_one_hot_feat, verbose=0)
print(f"\n--- 561-Feature + CNN 모델 테스트 결과 ---")
print(f"테스트 데이터 손실 (Loss): {loss_f:.4f}")
print(f"테스트 데이터 정확도 (Accuracy): {accuracy_f:.4f}") #


--- 561-Feature + CNN 모델 테스트 결과 ---
테스트 데이터 손실 (Loss): 0.1481
테스트 데이터 정확도 (Accuracy): 0.9589
